<a href="https://colab.research.google.com/github/21-Vaibhav/Major-project---24/blob/main/HAN_final_implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installing required libraries
!pip install transformers datasets torch torchtext
!pip install keras nltk


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 38.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset

# Loading the legal dataset
dataset = load_dataset("d0r1h/ILC")
print(dataset)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/368 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/36.5M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/17.7M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2058 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1015 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Title', 'Summary', 'Case'],
        num_rows: 2058
    })
    test: Dataset({
        features: ['Title', 'Summary', 'Case'],
        num_rows: 1015
    })
})


In [3]:
import nltk
from nltk.tokenize import sent_tokenize
from transformers import AutoTokenizer

nltk.download('punkt')

# Loading a tokenizer for preprocessing
tokenizer = AutoTokenizer.from_pretrained("t5-small")  # Pretrained summarization model

def preprocess_data(batch):
    tokenized_inputs = tokenizer(batch['Case'], truncation=True, padding=True, max_length=None)
    tokenized_outputs = tokenizer(batch['Summary'], truncation=True, padding=True, max_length=None)
    return {'input_ids': tokenized_inputs['input_ids'],
            'attention_mask': tokenized_inputs['attention_mask'],
            'labels': tokenized_outputs['input_ids']}

# Applying preprocessing
tokenized_dataset = dataset.map(preprocess_data, batched=True)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

Map:   0%|          | 0/2058 [00:00<?, ? examples/s]

Map:   0%|          | 0/1015 [00:00<?, ? examples/s]

In [4]:
import torch
torch.cuda.empty_cache()  # Clearing cache between training steps


In [5]:
from transformers import T5ForConditionalGeneration, Trainer, TrainingArguments

# Load pretrained T5 model
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    gradient_accumulation_steps=2,
    save_strategy="epoch",
    weight_decay=0.01,
    logging_dir="./logs",
    fp16=True,
    logging_steps=10
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset['train'],
    eval_dataset=tokenized_dataset['test']
)

# Fine-tune the model
trainer.train()


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

...

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
0,3.478700,3.244210
2,3.390600,3.100109


TrainOutput(global_step=771, training_loss=3.725583404882409, metrics={'train_runtime': 455.7241, 'train_samples_per_second': 13.548, 'train_steps_per_second': 1.692, 'total_flos': 834246864273408.0, 'train_loss': 3.725583404882409, 'epoch': 2.994174757281553})

In [17]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HANModel(nn.Module):
    def __init__(self, embed_size, hidden_size, vocab_size):
        super(HANModel, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.gru_sentence = nn.GRU(embed_size, hidden_size, bidirectional=True, batch_first=True)
        self.gru_document = nn.GRU(hidden_size * 2, hidden_size, bidirectional=True, batch_first=True)
        self.attention = nn.Linear(hidden_size * 2, 1)

    def forward(self, x):
        # Sentence-level GRU
        embedded_sentences = self.embedding(x)
        sentence_output, _ = self.gru_sentence(embedded_sentences)

        # Sentence attention
        attention_weights = F.softmax(self.attention(sentence_output), dim=1)
        sentence_vector = torch.sum(attention_weights * sentence_output, dim=1)

        # Document-level GRU
        document_output, _ = self.gru_document(sentence_vector.unsqueeze(1))

        # Document attention
        doc_attention_weights = F.softmax(self.attention(document_output), dim=1)
        document_vector = torch.sum(doc_attention_weights * document_output, dim=1)
        return document_vector

        print("document vector: ", document_vector)


In [7]:
class SummarizationModel(nn.Module):
    def __init__(self, pretrained_model, han_model):
        super(SummarizationModel, self).__init__()
        self.pretrained_model = pretrained_model
        self.han_model = han_model

    def forward(self, input_ids, attention_mask, labels):
        han_embeddings = self.han_model(input_ids)
        outputs = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        outputs.logits += han_embeddings  # Adding HAN output to pretrained logits
        return outputs


In [8]:
model.save_pretrained('./legal_summarizer')
tokenizer.save_pretrained('./legal_summarizer')


('./legal_summarizer/tokenizer_config.json',
 './legal_summarizer/special_tokens_map.json',
 './legal_summarizer/spiece.model',
 './legal_summarizer/added_tokens.json',
 './legal_summarizer/tokenizer.json')

In [9]:
import shutil

shutil.make_archive('legal_summarizer', 'zip', './legal_summarizer')


'/content/legal_summarizer.zip'

In [10]:
from google.colab import files

files.download('legal_summarizer.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
from transformers import pipeline, AutoTokenizer

# Load the saved model
summarizer = pipeline("summarization", model="./legal_summarizer", tokenizer="./legal_summarizer")
tokenizer = AutoTokenizer.from_pretrained("./legal_summarizer")

# Split text into token-limited chunks
def split_text_with_token_limit(text, tokenizer, max_tokens=512, overlap=50):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk_tokens, skip_special_tokens=True))
    return chunks

# Input text
input_text = "IN THE HIGH COURT OF DELHI AT NEW DELHI CRLREVP 774 2018  CRLMB1392 2018 Date of decision 03rd February 2021 G D KATARIA  Petitioner Through Mr Medhanshu Tripathi Advocate IN THE MATTER OF AVL LEASING  FINANCE LTD Respondent Through Mr Anuj Soni Advocate HON BLE MR JUSTICE SUBRAMONIUM PRASAD SUBRAMONIUM PRASAD J The instant revision petition is directed against the order dated 27082018 passed by the Additional Session Judge Special JudgeWest District Tis Hazari Court Delhi in Criminal Appeal being CA No 153 2018 whereby the Additional Session Judge has dismissed the appeal and has affirmed the order dated 26042018 passed by the Metropolitan Magistrate in CC No8073 2016 convicting the petitioner herein for offences punishable under section 138 of Negotiable Act 1881Provisio of NI Act read with Section 35713 of CrPC The facts leading to the instant revision petition are as under a The respondent is a leasing and financing company The respondent financed a bus for the petitioner by giving a loan In discharge of the liability the petitioner handed over three cheques drawn on Bank of Punjab Limited Rajouri Garden Delhi bearing number 327226 dated 13042003 for an amount of Rs 1 lakh number 327227 dated 28072003 for an amount of Rs 3 lakhs and number 327338 dated 27072003 for an amount of Rs 2 84 000 in favor of the respondent b When the respondent deposited these cheques they were returned as unpaid dishonored reason Funds c Notice as required under Section 138 of the NI Act was issued by the respondent calling upon the petitioner herein to make the payment within 15 days of receipt of the notice The payment was not received and a complaint was filed under Section 138 of the NI Act before the court of the Metropolitan Magistrate West District Tis Hazari Court d Before the Metropolitan Magistrate it was contended by the petitioner herein that he took a loan for purchasing a bus with registration number DL1 PA 5798 and at the time of taking the loan 36 blank signed cheques were given as security towards CRLREVP 774 2018 the repayment of the loan It is stated that in October 2002 the petitioner herein handed over the vehicle to the respondent company for getting the vehicle converted to CNG and entered into a new lease agreement but neither was the said vehicle returned to the petitioner herein nor were the accounts related to the hire purchase agreement settled It was stated before the Metropolitan Magistrate that as the vehicle is in the possession of the respondent herein and the cheque given by the petitioner accused had been misused It was stated before the Metropolitan Magistrate that the bus NoDL1 PA 5798 was re possessed by the respondent It was also stated the a No Objection Certificate dated 30082003 along with form 35 was issued by the issuing authority and the respondent herein had given a No objection with regard to the bus bearing NoDL1 PA 5798 The Metropolitan Magistrate after examining the documents found that the deposition of the accused petitioner herein is inconsistent with the evidence on record The Metropolitan Magistrate also found that in relation to bus number DL1 PA 5798 accused petitioner herein had deposed complainant respondent herein has already sold the bus but officer from Transport Department Delhi placed on record the RC which shows that the accused petitioner herein is the owner of the bus As per the RC bus number DL1 PA 5798 is registered in the name of the accused and NOC has been issued by hypothecatee on 10092003 The Metropolitan CRLREVP 774 2018 Magistrate held that the averments of the accused petitioner herein in relation to the sale of this vehicle despite payment of 13 installments is incorrect as the ownership of the vehicle still vests with the accused petitioner herein g The Metropolitan Magistrate after examining the documents came to the conclusion that this case pertains to the bus bearing registration number DL1 P 7279 h The Metropolitan Magistrate found that the accused petitioner herein and the complainant respondent herein had dealings in relation to two buses It was found that the petitioner had another bus bearing No DL1 P 7279 which was under the loan of the Motor  General Finance Ltd Company in the year 1995 The loan was taken over by the respondent company The Metropolitan Magistrate held that AR of the complainant deposed that a sum of Rs 8 Lakhs was lent to the accused in the year 2001 wherein certain payments were made to the accused and the remaining payment was made to MGF India Limited The Metropolitan Magistrate held that this was consistent with the testimony of the accused wherein he has admitted that he cleared his loan with MGF India Limited in 2001 CW 1 also deposed that one bus was in the name of the accused and the other bus was financed and the amount deposited by the complainant company was against the bus which was already in the name of the accusedThe Metropolitan Magistrate therefore held accused petitioner herein has not been able to rebut the presumption that the cheques had been paid for discharge of The Metropolitan Magistrate by an order dated 26042018 convicted the petitioner herein for offences under Section 138 any liability of the NI Act m The above said judgment was challenged before the Additional Session Judge in CA No 153 2018 After going through the records the Additional Session Judge upheld the judgment dated 02072018 passed by the Metropolitan Magistrate West District Tis Hazari Courts Delhi in CC No8073 2016 convicting the petitioner and also the order on sentence dated 12072018 passed by the Metropolitan Magistrate West District Tis Hazari Courts Delhi in CC No8073 2016 imposing simple imprisonment for two months period and fine of Rs 13 Lakhs n The Additional Session Judge after going through the material on record found that the three cheques were issued by the CRLREVP 774 2018 petitioner towards legally enforceable debt due to respondent Heard Mr Medhanshu Tripathi learned counsel appearing for the petitioner and Mr Anuj Soni learned counsel appearing for the respondent 4 Mr Medhanshu Tripathi learned counsel for the petitioner contends that the failure on part of the respondent to submit their books of accounts is fatal to the case of the respondent He would contend that the courts below ought to have drawn adverse inference against the respondent when the respondent deliberately did not produce the books of accounts even though the respondent had undertaken to produce the books of accounts 5 Mr Medhanshu Tripathi learned counsel for the petitioner would rely on the judgment of the Supreme Court in MS Narayana Menon v State of Kerala reported as 6 SCC 39 to contend that it was for the complainant respondent herein to produce the books of accounts and without producing the books of accounts it cannot be said that there was a debt due and subsisting and the cheques have been issued for discharge of debt 6 Mr Tripathi would further contend that the Metropolitan Magistrate had erred in relying upon the improved testimony of the respondent company He would further contend that the vehicle had been re possessed and sold and therefore nothing was due and payable On the other hand Mr Anuj Soni learned counsel for the respondent would support the support the judgments of the courts below to contend that all the points now being raised have already been dealt with by the Metropolitan Magistrate in his judgment and which has not been disturbed by the appellate Court Section 118 of the NI Act raises a presumption that a cheque is issued for consideration until the contrary is proved It is well settled CRLREVP 774 2018 position that the initial burden in this regard lies on the accused to prove the non existence of debt by bringing on record such facts and circumstances which would lead the court to believe the non existence of debt either by direct evidence or by preponderance of probabilities In the present case other than mere ipse dixit of the petitioner that there was no debt due and payable nothing is on record to show that the cheques were not issued for discharge of liability for the bus The second bus bearing registration No DL 1 PA 5798 stood in the name of the accused There is nothing to show that the liability for the first bus bearing registration No DL1 P 7279 has been discharged 10 The purpose of introducing Section 138 of the NI Act was to bring sanctity in commercial transactions In Dalmia CementLtd v Galaxy Traders  Agencies Ltd reported as 6 SCC 463 the Supreme Court observed as under to negotiable 3 The Act was enacted and Section 138 thereof incorporated with a specified object of making a special provision by incorporating a strict liability so far as the cheque a negotiable instrument is concerned The law law of commercial world legislated to facilitate the activities in trade and commerce making provision of giving sanctity to the instruments of credit which could be deemed to be convertible into money and easily passable from one person to another In the absence of such instruments including a cheque the trade and commerce activities in the present day world are likely to be adversely affected as it is impracticable for the trading community to carry on with it the bulk of the currency in force The negotiable instruments are in fact the instruments of credit being convertible on account of legality of being negotiated and are easily passable from one hand to CRLREVP 774 2018 another To achieve the objectives of the Act the legislature has in its wisdom thought it proper to make such provisions in the Act for conferring such privileges to the mercantile instruments contemplated under it and provide special penalties and procedure in case the obligations under the instruments are not discharged The laws relating to the Act are therefore required to be interpreted in the light of the objects intended to be achieved by it despite there being deviations from the general law and the procedure provided for the redressal of the grievances to the litigants Efforts to defeat the objectives of law by resorting to innovative measures and methods are to be discouraged lest it may affect the commercial and mercantile activities in a smooth and healthy manner ultimately affecting the economy of the country 12 Two courts below have looked into the entire records of the case and have come to the conclusion that the cheques have been given in discharge of debt The petitioner only seeks to take advantage of the fact that the respondent did not produce the books of accounts to rebut the initial presumption which was for the petitioner to show that the amount of loan taken by him and the amount that should be repaid in order to discharge the initial burden and the petitioner has failed to discharge the initial onus of 13 The scope of the revision petition under Sections 397 401 CrPC read with Section 482 CrPC is extremely narrow In State v Manimaran reported as 13 SCC 670 the Supreme Court observed as under 16 As held in State of Kerala v Puttumana Illath Jathavedan Namboodiri2 SCC 452  1999 SCC275 ordinarily it would not be appropriate CRLREVP 774 2018 In State of Haryana v Rajmal reported as 14 SCC 326 the Supreme Court observed as under the courts below recorded for the High Court to reappreciate the evidence and come to its own conclusion on the same when the evidence has already been appreciated by Magistrate as well as by the Sessions Court in appeal the concurrent findings of fact in our view the High Court was not right in interfering with the concurrent findings of fact arrived at by the courts below and the impugned order cannot be sustained in upsetting AP v Pituhuk the exercise of the High Court 14 In State Rao9 SCC 537  2001 SCC 642 this the revisional Court held jurisdiction of concurrent finding of the facts cannot be accepted when it was without any reference to the evidence on record or to the finding entered by the trial court and the appellate court regarding the evidence in view of the fact that revisional jurisdiction is basically supervisory in nature It has been also held by this Court in Amar Chand Agarwalla v Shanti Bose4 SCC 10  1973 SCC Cri 651  AIR 1973 SC 799 that the revisional jurisdiction of the High Court under Section 439 CrPC is to be exercised only in an exceptional case when there is a glaring defect in the procedure or there is a manifest error on a point of law resulting in a flagrant miscarriage of justice Report In State of Kerala v Puttumana Illath Jathavedan Namboodiri reported as 1999 2 SCC 452 the Supreme Court observed as under 5 Having examined the impugned judgment of the High CRLREVP 774 2018 Court and bearing in mind the contentions raised by the learned counsel for the parties we have no hesitation to come to the conclusion that in the case in hand the High Court has exceeded its revisional jurisdiction In its revisional jurisdiction the High Court can call for and examine the record of any proceedings for the purpose of satisfying itself as to the correctness legality or propriety of any finding sentence or order In other words the jurisdiction is one of supervisory jurisdiction exercised by the High Court for correcting miscarriage of justice But the said revisional power cannot be equated with the power of an appellate court nor can it be treated even as a second appellate jurisdiction Ordinarily therefore it would not be appropriate for the High Court to re appreciate the evidence and come to its own conclusion on the same when the evidence has already been appreciated by the Magistrate as well as the Sessions Judge in appeal unless any glaring feature is brought to the notice of the High Court which would otherwise tantamount to gross miscarriage of justice On scrutinizing the impugned judgment of the High Court from the aforesaid standpoint we have no hesitation to come to the conclusion that the High Court exceeded its jurisdiction in interfering with the conviction of the respondent by re appreciating the oral evidence The High Court also committed further error in not examining several items of evidence relied upon by the Additional Sessions Judge while confirming conviction of the respondent In this view of the matter the impugned judgment of the High Court is wholly unsustainable in law and we accordingly set aside the same The conviction and sentence of the respondent as passed by the Magistrate and affirmed by the Additional Sessions Judge in appeal is confirmed This appeal is allowed Bail bonds furnished stand cancelled The respondent must surrender to serve the sentence CRLREVP 774 2018 14 Having gone through the material on record this court does not find that the judgment of the courts below require any interference The learned counsel for the petitioner has not been able to demonstrate that the findings of the courts below are perverse The fact that the respondent did not file the books of accounts is not fatal to the case of the respondent It was open to the petitioner to produce his books of accounts to rebut the presumption and bring out a prima facie case that there was no debt due and payable on the date the cheques were dishonoured The petitioner has not been able to discharge the initial burden on him to rebut the presumption under Section 118 of the NI Act The fact that the petitioner took financial assistance from the respondent is admitted The petitioner has not been able to show as to how there was no subsisting debt on the date when the cheques were dishonoured due to insufficiency of funds 15 No case has been made out which would warrant interference under 16 Accordingly the revision petition is dismissed along with the pending Section 397 401 CrPC application FEBRUARY 03 2021 SUBRAMONIUM PRASAD J CRLREVP 774 2018"

# Split into manageable chunks
text_chunks = split_text_with_token_limit(input_text, tokenizer)

# Summarize each chunk
summaries = []
for chunk in text_chunks:
    input_length = len(tokenizer(chunk, return_tensors="pt", truncation=True)["input_ids"][0])
    dynamic_max_length = min(700, input_length // 2)  # Adjust max_length dynamically
    summary = summarizer(chunk, max_length=dynamic_max_length, min_length=dynamic_max_length // 2, do_sample=False)
    summaries.append(summary[0]['summary_text'])

# Combine summaries
final_summary = ' '.join(summaries)
print("Summary:", final_summary)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Token indices sequence length is longer than the specified maximum sequence length for this model (3499 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


Summary: the instant revision petition is directed against the order dated 27082018 passed by the Additional Session Judge Special JudgeWest District Tis Hazari Court Delhi in Criminal Appeal being CA No 153 2018 . the respondent handed over three cheques drawn on Bank of Punjab Limited Rajouri Garden Delhi bearing number 327226 dated 13042003 for an amount of Rs 3 lakhs . a complaint was filed under Section 138 of the NI Act before the court of the Metropolitan Magistrate west district Tishazari vs. the petitioner is a leasing and financing company The respondent financed a bus with registration number DL1 PA 5798 and at the time of taking the loan 36 blank signed cheques were handed as unpaid dishonored reason Funds c Notice as required under the CC No8073 2016 . the petitioner herein took a loan for purchasing a bus with registration number DL1 PA 5798 and at the time of taking the loan 36 blank signed cheques were given as security towards CRLREVP 774 2018 the repayment of the loan

In [14]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification

# Load the legal summarizer model
summarizer = pipeline("summarization", model="./legal_summarizer", tokenizer="./legal_summarizer")
tokenizer = AutoTokenizer.from_pretrained("./legal_summarizer")

# Load the Legal-BERT model for outcome prediction
bert_model_name = "nlpaueb/legal-bert-base-uncased"  # Replace with the desired Legal-BERT model
bert_tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
bert_model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)

# Function to split text into manageable chunks
def split_text_with_token_limit(text, tokenizer, max_tokens=512, overlap=50):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk_tokens, skip_special_tokens=True))
    return chunks

# Function to generate summary and predict outcome
def predict_outcome(case_details):
    # Step 1: Split input text into chunks
    text_chunks = split_text_with_token_limit(case_details, tokenizer)

    # Step 2: Summarize each chunk
    summaries = []
    for chunk in text_chunks:
        input_length = len(tokenizer(chunk, return_tensors="pt", truncation=True)["input_ids"][0])
        dynamic_max_length = min(700, input_length // 2)  # Adjust max_length dynamically
        summary = summarizer(chunk, max_length=dynamic_max_length, min_length=dynamic_max_length // 2, do_sample=False)
        summaries.append(summary[0]['summary_text'])

    # Combine summaries
    final_summary = ' '.join(summaries)
    print("Summary Generated:\n", final_summary)

    # Step 3: Use Legal-BERT for outcome prediction
    inputs = bert_tokenizer(final_summary, return_tensors="pt", truncation=True, max_length=512)
    outputs = bert_model(**inputs)
    prediction = outputs.logits.argmax(-1).item()

    # Define label mapping (adjust based on your fine-tuned labels)
    label_mapping = {0: "Outcome: Favor of petitioner", 1: "Outcome: Favor of respondent"}
    outcome = label_mapping.get(prediction, "Outcome: Unknown")
    print("Predicted Outcome:\n", outcome)

# Input: Case details
case_details="Criminal AppealNo 10115 Against the judgment and order of conviction and sentence dated 27072015 by the learned Sessions Judge Lohardaga in Sessions Trial No 413 1Arti  Aarti  Sukar Kumari 2Jama  Sushil Munda 3Birsa Munda  Runka Munda 4Boda Munda  Suman Munda   Appellants The State of Jharkhand For the Appellant For the Respondent Mr Birendra Kumar Advocate Mr Vijay Kumar Sinha Advocate Mr Shekhar Sinha PP Present HON BLE MR JUSTICE RONGON MUKHOPADHYAY HONBLE MR JUSTICE RAJESH KUMAR Heard Mr Birendra Kumar learned counsel for the appellants and Mr Shekhar Sinha learned PP for the State This appeal is directed against the judgment and order of conviction and sentence dated 27072015passed by the learned Sessions Judge Lohardaga in Sessions Trial No 44 of 2013 whereby and whereunder the appellants have been convicted for the offences punishable under Sections 302 34 and 201 34 of IPC and have been sentenced to undergo SI for life and a fine of Rs 10 000 for the offence under Section 302 34 of IPC No separate sentence has been passed for the offence under Section 201 34 of IPC The prosecution story arising out of an FIR instituted by Jairam Munda is to the effect that the son of the informant had gone to his in laws place on 05022013 at village Panchpadwa in the district of Lohardaga On 06022013 at about 8 AM he came to know that his son has been murdered by Boda Munda  Suman Munda Birsa Munda  Runka Munda Jama Munda  Sushil Munda and Arti Kumari  Sukar Kumari At this he along with his another son reached Panchpadwa where he found his son having been murdered and thrown at Panchpadwa Tongri The reason for the occurrence is that the son of the 2 informant Mansukh Munda used to remain sick and in spite of being treated at various places he was not getting well It has been alleged that the informant and his family members were thinking of taking Mansukh Munda to Devas for treatment and this intention was disclosed to his brother Hadu Munda and on this issue there appeared a fissure in the relationship between the families of the informant and Hadu Munda It has been alleged that the family of Hadu Munda left for Panchpadwa The informant had sent his son to bring them back but they did not return On 28012013 the son of the informant Markas Munda had died and there was friction between both the sides for practicing witchcraft which resulted in the occurrence The informant could came to know that his son Markas Munda had gone to the house of Bauda Munda on 05022013 and near the door stop Arti Kumari had started assaulting him with fists slaps and danda It has been alleged that all the accused persons had dragged Markas Munda towards the jungle and assaulted him on his head with lathi and stones which resulted in his death It has also been stated that his relation Baura Munda has seen the occurrence Based on the aforesaid allegations Jobang P S Case No 2 of 2013 was instituted under Section 302  201 34 of IPC in which after investigation charge sheet was submitted leading to taking of cognizance and after the case was committed to the court of Sessions charge was framed for the offences punishable under Sections 302 34  201 34 of IPC and under Section 3 of the Prevention of Witch Practices Act 1999 to which the accused pleaded not guilty and claimed to be tried In course of trial 7 witnesses were examined by the prosecution PW 1 Boda Munda  Bauda Munda has stated that the incident is of 6 months back He had gone to Kisko to prepare his Aadhar Card and he returned back to his house from Kisko at about 8 PM He has stated that while returning home he has seen his son in law Markas Munda being assaulted with lathi and danda by 4 5 persons He has further stated that after assaulting him he was dragged towards the forest where he was hit by a big stone leading to his death He has further stated that the persons involved are Birsa Munda  Runka Munda Bora Munda  Suman Munda Jama Munda  Sushil Munda PW 2 Sohrai Munda has stated that on the date of occurrence 3 and Aarti Kumari  Sukar Kumari He has stated that thereafter on account of fear he had gone to his residence In cross examination this witness has stated that it was dark when the incident had taken place He has further deposed that due to darkness persons who were standing at a distance could not be recognized He has also stated that he does not know as to when his son in law had come to his house After coming from Kisko he had straightway gone to his house He has further stated that all the accused are from his village with whom this witness is not on very friendly terms Replying to a court question this witness has stated that he was at a distance of 150 yards from the place of occurrence he along with PW 1 was returning from Kisko to their respective homes On reaching the village they heard the sound of shouting and when they reached near the house of PW 1 they had seen Markas Munda being assaulted by the accused persons He has further stated that the accused persons took Markas Munda towards Tongri and thereafter Markas Munda was assaulted on his head with a big stone and he and PW 1 fled away from the place of occurrence and reached their respective homes He has stated that on the next date the father of Markas Munda was informed In cross examination this witness has stated that it was total darkness at the time of returning home from Kisko Block He has admitted that there is no electricity in his village He has further stated that while entering his village his house is situated before the house of PW 1 He has further stated that the incident was never disclosed by them to the villagers He has also deposed that he had not seen any person in the place of occurrence occurrence he was in the house and on next date Sohrai Munda informed on phone that his elder brother has been done to death He has also stated that on hearing such information he had gone to village Panchpadwa and had seen the dead body of his brother Sohrai had disclosed to him about the accused persons committing the murder of his brother He has also stated that the reason for the occurrence is that Devas had told him that it was on account of his uncle Hadu Munda that his brother Mansukh Munda was sick PW 3 Ram Nath Munda has stated that on the date of 4 PW 4 Dr Pranav Kumar was posted as a Medical Officer on 07022013 in the Department of Leprosy where he conducted autopsy on the dead body of Markas Munda This witness has found the following External Injuries Lacerated wound on left side of forehead size 2 inch x 1 inch x up to bone deep Fracture of left mandible iii Extraction of upper four teeth iv Blood stain on nose and ear Internal Injuries A Head and Neck Brain matter lacerated and pale on section B Thoracic cavity Lungs intact and pale on section heart Intact right chamber full and left chamber empty C Abdominal cavity Liver spleen and kidneys were intact and pale on cut section Stomach contains semi digested food particle and gas large intestine contains faecal matter All above injuries were ante morten in nature and time elapsed since death 6 to 36 hours The cause of death according to this witness was due to Haemorrhage and shock caused by hard and blunt substance This witness has proved the post mortem report which has been marked as Exhibit 1 PW 5 Dedo Munda is a hearsay witness PW 6 Jairam Munda is the informant who has stated that on the date of occurrence he was at his home and his son Markas Munda had gone to his in laws place at village Panchpadwa He has also stated that his son had gone to call the brother of the informant namely Hadu Munda He has also stated that his son was assaulted and done to death by the accused persons The reasons for the occurrence according to this witness was that he and his brother were not on good terms In cross examination it has been stated that there was no enmity with the accused Bora Munda Jama Munda and Birsa Munda and they were on friendly terms with him 5 It has been stated by the learned counsel for the appellants that the 11 PW 7 Supa Kongari has deposed that on 06022013 he was posted as Sub Inspector on Jobang Police Station He has recorded the Fard Beyan of Jairam Munda which has been proved and marked as Exhibit 2 The formal FIR is in his hand writing and signature which has been marked as Exhibit 3 He has further stated that after taking over investigation he had recorded the re statement of the informant and had thereafter taken the statement of Baura Munda Sohrai Munda Dedo Munda and Ram Nath Munda This witness has inspected the place of occurrence which is at village Panchpadwa Tongri which is an isolated place surrounded by jungle and bushes On 17022013 he had obtained the post mortem report and on direction of his superior officials he had submitted charge sheet In cross examination this witness has stated that he had not seized any article during investigation He has also stated that the house of Bauda Munda is at a distance of half km from the place of occurrence On a court question he has stated that inquest report has been misplaced and he also could not recover the big stone which was used in the commission of the murder prosecution has miserably failed to prove its case beyond all reasonable doubt Learned counsel submits that save and except PW 1 and 2 there are no eye witnesses to the occurrence and the evidence of PW 1 and PW 2 suffers from major discrepancies which itself makes the identification of the appellants as the perpetrators of the offence doubtful Learned counsel submits that the PW 1 and PW 2 had stated that it was a dark night and there is no electricity in the village and as such it would have been absolutely impossible to have identified the assailants Learned counsel further while referring to the evidence of PW 1 submits that the conduct of PW 1 is improbable and unbelievable if his version is taken to be true to the effect that he claims to have seen the accused persons committing the murder of his son in law but neither had he tried to save him nor did he inform this matter to any of the villagers immediately Learned counsel also submits that as per PW 2 the incident is said to have taken place near the house of PW 1 but the IO has disclosed that the distance between the place of occurrence and the house of PW 1 is about half a km He further submits that nothing 6 has been stated as to whether PW 1 and PW 2 had followed them which further nullifies the story created by PW 1 and PW 2 to implicate the appellants Learned counsel submits that as per PW 1 both the sides were on inimical terms and hence the false implication of the appellant cannot be ruled out On such parameters learned counsel for the appellants has stated that the present appeal be allowed 13 Mr Shekhar Sinha learned PP for the State on the other hand while relying on the evidence of PW 1 and PW 2 has stated that they are the eye witnesses of the occurrence and their evidence is cogent convincing and believable and therefore conviction of the appellants under Section 302 34 and 201 34 of IPC has rightly been done He therefore prays that the present appeal be dismissed 14 We have heard the rival submissions and have also gone through the Lower Court Records The prosecution has tried to build an edifice based on the evidence of PW 1 and PW 2 who have stated to have seen the occurrence relating to the son of the informant namely Markas Munda being subjected to assault by the appellants with stone and tangi However the edifice crumbles once the cross examination of PW 1 and PW 2 is taken into consideration PW 1 in very categorical terms has stated in his cross examination that it was difficult for the persons to be recognized who were standing at a distance on account of the darkness He has further admitted that it was a dark night According to the evidence of PW 1 he out of fear had returned back to his residence which conduct appears to be totally contrary to what one would expect in such a situation when the son in law of the PW 1 is being done to death and he neither informs the villagers nor raises any alarm but rushes to his house and stays put On a court question he has stated that from the place of occurrence he was at a distance of 150 feet and considering the darkness which had engulfed the area and the distance from where the incident is said to have been seen by PW 1 it totally nullifies the prosecution case with respect to the identification of the appellants being the assailants Similarly the evidence of PW 2 is fraught with major discrepancies as he has stated that he had not seen any 7 person in the place of occurrence and that he had never disclosed the incident to the villagers This witness has also admitted that there is no electricity in the village He has also stated about total darkness when he was returning home from Kisko Block The identification of the appellants by PW 1 also is nullified if the distance between the place of occurrence and the house of PW 1 is taken into consideration as disclosed by PW 7 the Investigating Officer which is about a half km and coupled with the darkness the identification of the appellants by PW 2 therefore seems to be an absurd story created by PW 2 It is to be noted herein that PW 1 has stated about he being on inimical terms with the appellants and therefore the false implication of the appellants also cannot be ruled out The Investigating Officer has failed to seize the weapon of assault or the stone with which the deceased was purportedly done to death The evidence of the other witnesses do not energise the prosecution case and the falsity of the allegations levelled against the appellants lie threadbare on a dissection of the evidence of the so called eye witnesses PW 1 and PW 2 as has been taken note of earlier The prosecution therefore having miserably failed to prove its case beyond all reasonable doubt the appellants deserves to be acquitted from the charges against them In such circumstances this appeal stands allowed and the impugned judgment and order of conviction and sentence dated 27072015 and 31072015 respectively passed by the learned Sessions Judge Lohardaga in Sessions Trial No 413 are hereby set aside Since the appellant no 1 is on bail she is discharged from the liability of her bail bonds As regards appellant nos 2 to 4 are concerned they are directed to be released immediately and forthwith if not wanted in any other case This appeal stands allowed Rongon Mukhopadhyay J Rajesh Kumar J Jharkhand High Court at Ranchi The 28th day of July 2021 RShekhar NAFR Cp3"
predict_outcome(case_details)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/222k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at nlpaueb/legal-bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (3476 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


Summary Generated:
 the appeal is directed against the judgment and order of conviction and sentence dated 27072015passed by the learned Sessions Judge Lohardaga in Sessions Trial No 413 1Arti Aarti Sukar Kumari 2Jama Sushil Munda 3Birsa Munda Runka Munda 4Boda Mundo Suman Munda Appellants The State of Jharkhand For the Respondent Mr Birendra Kumar Advocate Mr Vijay Kumar Sinha PP Present HON BLE MR JUSTICE RAJESH KUMAR and 201 34 of IPC and have been sentenced to undergo SI for life and a fine of Rs 10 000 for the offences punishable under Section 302 34 of 2013 he came to know that the son of the 2 informant had gone to his in laws place on 06022013 he was not getting well . the appellants have been convicted for the offense under Section 201, 34 of the IPC . he and his another son reached Panchpadwa in the district of . alleged that the informant and his family were thinking of taking Mansukh Munda to Devas for treatment and this intention was disclosed to his brother Hadu Munda and

In [21]:
import numpy as np
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./legal_summarizer")
tokenizer = AutoTokenizer.from_pretrained("./legal_summarizer")

# Enable output of hidden states
model.config.output_hidden_states = True

# Split text into token-limited chunks
def split_text_with_token_limit(text, tokenizer, max_tokens=512, overlap=50):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk_tokens, skip_special_tokens=True))
    return chunks

# Summarize and save document vectors
def summarize_and_save_vectors(text_chunks, model, tokenizer, vector_file_path):
    summaries = []
    all_vectors = []

    for idx, chunk in enumerate(text_chunks):
        # Tokenize chunk
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True)

        # Forward pass through the encoder to extract hidden states
        encoder_outputs = model.encoder(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            return_dict=True,
            output_hidden_states=True,
        )

        # Extract the encoder's last hidden state and compute the document vector
        document_vector = encoder_outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()  # Mean pooling
        all_vectors.append(document_vector)
        print(f"Processed chunk {idx + 1}/{len(text_chunks)}")

        # Generate summary using the full model
        summary_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            min_length=50,
            do_sample=False,
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

    # Save all document vectors to a file
    np.save(vector_file_path, np.vstack(all_vectors))  # Stack and save all vectors
    print(f"Document vectors saved to {vector_file_path}")

    return summaries

# Input text
input_text = "Criminal AppealNo 10115 Against the judgment and order of conviction and sentence dated 27072015 by the learned Sessions Judge Lohardaga in Sessions Trial No 413 1Arti  Aarti  Sukar Kumari 2Jama  Sushil Munda 3Birsa Munda  Runka Munda 4Boda Munda  Suman Munda   Appellants The State of Jharkhand For the Appellant For the Respondent Mr Birendra Kumar Advocate Mr Vijay Kumar Sinha Advocate Mr Shekhar Sinha PP Present HON BLE MR JUSTICE RONGON MUKHOPADHYAY HONBLE MR JUSTICE RAJESH KUMAR Heard Mr Birendra Kumar learned counsel for the appellants and Mr Shekhar Sinha learned PP for the State This appeal is directed against the judgment and order of conviction and sentence dated 27072015passed by the learned Sessions Judge Lohardaga in Sessions Trial No 44 of 2013 whereby and whereunder the appellants have been convicted for the offences punishable under Sections 302 34 and 201 34 of IPC and have been sentenced to undergo SI for life and a fine of Rs 10 000 for the offence under Section 302 34 of IPC No separate sentence has been passed for the offence under Section 201 34 of IPC The prosecution story arising out of an FIR instituted by Jairam Munda is to the effect that the son of the informant had gone to his in laws place on 05022013 at village Panchpadwa in the district of Lohardaga On 06022013 at about 8 AM he came to know that his son has been murdered by Boda Munda  Suman Munda Birsa Munda  Runka Munda Jama Munda  Sushil Munda and Arti Kumari  Sukar Kumari At this he along with his another son reached Panchpadwa where he found his son having been murdered and thrown at Panchpadwa Tongri The reason for the occurrence is that the son of the 2 informant Mansukh Munda used to remain sick and in spite of being treated at various places he was not getting well It has been alleged that the informant and his family members were thinking of taking Mansukh Munda to Devas for treatment and this intention was disclosed to his brother Hadu Munda and on this issue there appeared a fissure in the relationship between the families of the informant and Hadu Munda It has been alleged that the family of Hadu Munda left for Panchpadwa The informant had sent his son to bring them back but they did not return On 28012013 the son of the informant Markas Munda had died and there was friction between both the sides for practicing witchcraft which resulted in the occurrence The informant could came to know that his son Markas Munda had gone to the house of Bauda Munda on 05022013 and near the door stop Arti Kumari had started assaulting him with fists slaps and danda It has been alleged that all the accused persons had dragged Markas Munda towards the jungle and assaulted him on his head with lathi and stones which resulted in his death It has also been stated that his relation Baura Munda has seen the occurrence Based on the aforesaid allegations Jobang P S Case No 2 of 2013 was instituted under Section 302  201 34 of IPC in which after investigation charge sheet was submitted leading to taking of cognizance and after the case was committed to the court of Sessions charge was framed for the offences punishable under Sections 302 34  201 34 of IPC and under Section 3 of the Prevention of Witch Practices Act 1999 to which the accused pleaded not guilty and claimed to be tried In course of trial 7 witnesses were examined by the prosecution PW 1 Boda Munda  Bauda Munda has stated that the incident is of 6 months back He had gone to Kisko to prepare his Aadhar Card and he returned back to his house from Kisko at about 8 PM He has stated that while returning home he has seen his son in law Markas Munda being assaulted with lathi and danda by 4 5 persons He has further stated that after assaulting him he was dragged towards the forest where he was hit by a big stone leading to his death He has further stated that the persons involved are Birsa Munda  Runka Munda Bora Munda  Suman Munda Jama Munda  Sushil Munda PW 2 Sohrai Munda has stated that on the date of occurrence 3 and Aarti Kumari  Sukar Kumari He has stated that thereafter on account of fear he had gone to his residence In cross examination this witness has stated that it was dark when the incident had taken place He has further deposed that due to darkness persons who were standing at a distance could not be recognized He has also stated that he does not know as to when his son in law had come to his house After coming from Kisko he had straightway gone to his house He has further stated that all the accused are from his village with whom this witness is not on very friendly terms Replying to a court question this witness has stated that he was at a distance of 150 yards from the place of occurrence he along with PW 1 was returning from Kisko to their respective homes On reaching the village they heard the sound of shouting and when they reached near the house of PW 1 they had seen Markas Munda being assaulted by the accused persons He has further stated that the accused persons took Markas Munda towards Tongri and thereafter Markas Munda was assaulted on his head with a big stone and he and PW 1 fled away from the place of occurrence and reached their respective homes He has stated that on the next date the father of Markas Munda was informed In cross examination this witness has stated that it was total darkness at the time of returning home from Kisko Block He has admitted that there is no electricity in his village He has further stated that while entering his village his house is situated before the house of PW 1 He has further stated that the incident was never disclosed by them to the villagers He has also deposed that he had not seen any person in the place of occurrence occurrence he was in the house and on next date Sohrai Munda informed on phone that his elder brother has been done to death He has also stated that on hearing such information he had gone to village Panchpadwa and had seen the dead body of his brother Sohrai had disclosed to him about the accused persons committing the murder of his brother He has also stated that the reason for the occurrence is that Devas had told him that it was on account of his uncle Hadu Munda that his brother Mansukh Munda was sick PW 3 Ram Nath Munda has stated that on the date of 4 PW 4 Dr Pranav Kumar was posted as a Medical Officer on 07022013 in the Department of Leprosy where he conducted autopsy on the dead body of Markas Munda This witness has found the following External Injuries Lacerated wound on left side of forehead size 2 inch x 1 inch x up to bone deep Fracture of left mandible iii Extraction of upper four teeth iv Blood stain on nose and ear Internal Injuries A Head and Neck Brain matter lacerated and pale on section B Thoracic cavity Lungs intact and pale on section heart Intact right chamber full and left chamber empty C Abdominal cavity Liver spleen and kidneys were intact and pale on cut section Stomach contains semi digested food particle and gas large intestine contains faecal matter All above injuries were ante morten in nature and time elapsed since death 6 to 36 hours The cause of death according to this witness was due to Haemorrhage and shock caused by hard and blunt substance This witness has proved the post mortem report which has been marked as Exhibit 1 PW 5 Dedo Munda is a hearsay witness PW 6 Jairam Munda is the informant who has stated that on the date of occurrence he was at his home and his son Markas Munda had gone to his in laws place at village Panchpadwa He has also stated that his son had gone to call the brother of the informant namely Hadu Munda He has also stated that his son was assaulted and done to death by the accused persons The reasons for the occurrence according to this witness was that he and his brother were not on good terms In cross examination it has been stated that there was no enmity with the accused Bora Munda Jama Munda and Birsa Munda and they were on friendly terms with him 5 It has been stated by the learned counsel for the appellants that the 11 PW 7 Supa Kongari has deposed that on 06022013 he was posted as Sub Inspector on Jobang Police Station He has recorded the Fard Beyan of Jairam Munda which has been proved and marked as Exhibit 2 The formal FIR is in his hand writing and signature which has been marked as Exhibit 3 He has further stated that after taking over investigation he had recorded the re statement of the informant and had thereafter taken the statement of Baura Munda Sohrai Munda Dedo Munda and Ram Nath Munda This witness has inspected the place of occurrence which is at village Panchpadwa Tongri which is an isolated place surrounded by jungle and bushes On 17022013 he had obtained the post mortem report and on direction of his superior officials he had submitted charge sheet In cross examination this witness has stated that he had not seized any article during investigation He has also stated that the house of Bauda Munda is at a distance of half km from the place of occurrence On a court question he has stated that inquest report has been misplaced and he also could not recover the big stone which was used in the commission of the murder prosecution has miserably failed to prove its case beyond all reasonable doubt Learned counsel submits that save and except PW 1 and 2 there are no eye witnesses to the occurrence and the evidence of PW 1 and PW 2 suffers from major discrepancies which itself makes the identification of the appellants as the perpetrators of the offence doubtful Learned counsel submits that the PW 1 and PW 2 had stated that it was a dark night and there is no electricity in the village and as such it would have been absolutely impossible to have identified the assailants Learned counsel further while referring to the evidence of PW 1 submits that the conduct of PW 1 is improbable and unbelievable if his version is taken to be true to the effect that he claims to have seen the accused persons committing the murder of his son in law but neither had he tried to save him nor did he inform this matter to any of the villagers immediately Learned counsel also submits that as per PW 2 the incident is said to have taken place near the house of PW 1 but the IO has disclosed that the distance between the place of occurrence and the house of PW 1 is about half a km He further submits that nothing 6 has been stated as to whether PW 1 and PW 2 had followed them which further nullifies the story created by PW 1 and PW 2 to implicate the appellants Learned counsel submits that as per PW 1 both the sides were on inimical terms and hence the false implication of the appellant cannot be ruled out On such parameters learned counsel for the appellants has stated that the present appeal be allowed 13 Mr Shekhar Sinha learned PP for the State on the other hand while relying on the evidence of PW 1 and PW 2 has stated that they are the eye witnesses of the occurrence and their evidence is cogent convincing and believable and therefore conviction of the appellants under Section 302 34 and 201 34 of IPC has rightly been done He therefore prays that the present appeal be dismissed 14 We have heard the rival submissions and have also gone through the Lower Court Records The prosecution has tried to build an edifice based on the evidence of PW 1 and PW 2 who have stated to have seen the occurrence relating to the son of the informant namely Markas Munda being subjected to assault by the appellants with stone and tangi However the edifice crumbles once the cross examination of PW 1 and PW 2 is taken into consideration PW 1 in very categorical terms has stated in his cross examination that it was difficult for the persons to be recognized who were standing at a distance on account of the darkness He has further admitted that it was a dark night According to the evidence of PW 1 he out of fear had returned back to his residence which conduct appears to be totally contrary to what one would expect in such a situation when the son in law of the PW 1 is being done to death and he neither informs the villagers nor raises any alarm but rushes to his house and stays put On a court question he has stated that from the place of occurrence he was at a distance of 150 feet and considering the darkness which had engulfed the area and the distance from where the incident is said to have been seen by PW 1 it totally nullifies the prosecution case with respect to the identification of the appellants being the assailants Similarly the evidence of PW 2 is fraught with major discrepancies as he has stated that he had not seen any 7 person in the place of occurrence and that he had never disclosed the incident to the villagers This witness has also admitted that there is no electricity in the village He has also stated about total darkness when he was returning home from Kisko Block The identification of the appellants by PW 1 also is nullified if the distance between the place of occurrence and the house of PW 1 is taken into consideration as disclosed by PW 7 the Investigating Officer which is about a half km and coupled with the darkness the identification of the appellants by PW 2 therefore seems to be an absurd story created by PW 2 It is to be noted herein that PW 1 has stated about he being on inimical terms with the appellants and therefore the false implication of the appellants also cannot be ruled out The Investigating Officer has failed to seize the weapon of assault or the stone with which the deceased was purportedly done to death The evidence of the other witnesses do not energise the prosecution case and the falsity of the allegations levelled against the appellants lie threadbare on a dissection of the evidence of the so called eye witnesses PW 1 and PW 2 as has been taken note of earlier The prosecution therefore having miserably failed to prove its case beyond all reasonable doubt the appellants deserves to be acquitted from the charges against them In such circumstances this appeal stands allowed and the impugned judgment and order of conviction and sentence dated 27072015 and 31072015 respectively passed by the learned Sessions Judge Lohardaga in Sessions Trial No 413 are hereby set aside Since the appellant no 1 is on bail she is discharged from the liability of her bail bonds As regards appellant nos 2 to 4 are concerned they are directed to be released immediately and forthwith if not wanted in any other case This appeal stands allowed Rongon Mukhopadhyay J Rajesh Kumar J Jharkhand High Court at Ranchi The 28th day of July 2021 RShekhar NAFR Cp3"  # Replace with your input

# Split into manageable chunks
text_chunks = split_text_with_token_limit(input_text, tokenizer)

# File path for saving document vectors
vector_file_path = "document_vectors.npy"

# Generate summaries and save document vectors
summaries = summarize_and_save_vectors(text_chunks, model, tokenizer, vector_file_path)

# Combine summaries
final_summary = ' '.join(summaries)
print("Final Summary:", final_summary)


Token indices sequence length is longer than the specified maximum sequence length for this model (3476 > 512). Running this sequence through the model will result in indexing errors


Processed chunk 1/8
Processed chunk 2/8
Processed chunk 3/8
Processed chunk 4/8
Processed chunk 5/8
Processed chunk 6/8
Processed chunk 7/8
Processed chunk 8/8
Document vectors saved to document_vectors.npy
Final Summary: The prosecution story arising out of an FIR instituted by Jairam Munda is to the effect that the son of the informant had gone to his in laws place on 05022013 at about 8 AM he came to know that his son has been murdered by Boda Munda Suman Munda Birsa Munda Birsa Munda Birsa Munda and Arti Kumari Sukar Kumari The reason for the occurrence is that the son of the informant had gone to his in laws place on  In the case of Boda Munda Bauda Munda, the accused pleaded not guilty and claimed to be tried. The case was instituted under Section 302 201 34 of IPC and under Section 3 of the Prevention of Witchcraft Act 1999 to which the accused pleaded not guilty and claimed to be tried. The case was instituted under Section 302 201 34 of IPC and under Section 3 of the Preventio

In [24]:
import numpy as np
data = np.load('document_vectors.npy')
print(data)


[[-0.05999306  0.03267509 -0.00504601 ... -0.03562237  0.01910193
  -0.02501289]
 [-0.04416534  0.00118581  0.01078228 ... -0.01426361 -0.01111744
   0.00498432]
 [-0.0562905   0.03273813  0.00199961 ... -0.03898798  0.00638766
  -0.02416219]
 ...
 [-0.05377011  0.05086422 -0.01786188 ... -0.01765547 -0.02556029
   0.03546873]
 [-0.06902139  0.05918095 -0.02454863 ... -0.01458786 -0.00588951
   0.0558161 ]
 [-0.07595741  0.07246009 -0.02833497 ... -0.01408217  0.00820388
   0.02522478]]


In [28]:
import numpy as np
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./legal_summarizer")
tokenizer = AutoTokenizer.from_pretrained("./legal_summarizer")

# Enable output of hidden states and attention weights
model.config.output_hidden_states = True
model.config.output_attentions = True  # Enable attention outputs

# Split text into token-limited chunks
def split_text_with_token_limit(text, tokenizer, max_tokens=512, overlap=50):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk_tokens, skip_special_tokens=True))
    return chunks

# Save words with attention weights and print the document vector and attention scores
def save_and_print_vectors_and_attention(text_chunks, model, tokenizer, vector_file_path, attention_file_path):
    all_vectors = []
    all_attention_weights = []  # To store attention weights
    summaries = []

    for idx, chunk in enumerate(text_chunks):
        # Tokenize chunk
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True)

        # Forward pass through the encoder to extract hidden states and attention weights
        outputs = model.encoder(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            return_dict=True,
            output_hidden_states=True,
            output_attentions=True,  # Get attention weights
        )

        # Extract the encoder's last hidden state and attention weights
        document_vector = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()  # Mean pooling
        all_vectors.append(document_vector)

        # Extract attention weights from the output
        attentions = outputs.attentions  # List of attention weights for each layer
        # We'll use the attention from the last layer for simplicity
        last_attention = attentions[-1]  # (batch_size, num_heads, seq_len, seq_len)

        # Aggregate attention scores across all attention heads
        attention_scores = last_attention.mean(dim=1).squeeze().detach().cpu().numpy()  # Average across heads

        # Map tokens to words
        tokens = tokenizer.tokenize(chunk)
        word_attention_map = {}

        for i, token in enumerate(tokens):
            # Here we associate each token with its corresponding attention score
            word_attention_map[token] = attention_scores[i]

        all_attention_weights.append(word_attention_map)

        # Generate summary using the full model
        summary_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            min_length=50,
            do_sample=False,
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

        print(f"Processed chunk {idx + 1}/{len(text_chunks)}")

    # Save all document vectors to a file
    np.save(vector_file_path, np.vstack(all_vectors))  # Stack and save all vectors
    print(f"Document vectors saved to {vector_file_path}")

    # Save the attention weights to a text file
    with open(attention_file_path, 'w') as f:
        for idx, attention_map in enumerate(all_attention_weights):
            f.write(f"Attention for chunk {idx + 1}:\n")
            for token, score in attention_map.items():
                f.write(f"Token: {token}, Attention Score: {score}\n")
            f.write("\n")

    return summaries

# Print the content of the saved document vector file
def print_document_vectors(vector_file_path):
    print("\nDocument Vectors:")
    vectors = np.load(vector_file_path)
    print(vectors)

# Print the content of the saved attention weights file
def print_attention_weights(attention_file_path):
    print("\nAttention Weights:")
    with open(attention_file_path, 'r') as f:
        print(f.read())

# Input text
input_text = "Criminal AppealNo 10115 Against the judgment and order of conviction and sentence dated 27072015 by the learned Sessions Judge Lohardaga in Sessions Trial No 413 1Arti  Aarti  Sukar Kumari 2Jama  Sushil Munda 3Birsa Munda  Runka Munda 4Boda Munda  Suman Munda   Appellants The State of Jharkhand For the Appellant For the Respondent Mr Birendra Kumar Advocate Mr Vijay Kumar Sinha Advocate Mr Shekhar Sinha PP Present HON BLE MR JUSTICE RONGON MUKHOPADHYAY HONBLE MR JUSTICE RAJESH KUMAR Heard Mr Birendra Kumar learned counsel for the appellants and Mr Shekhar Sinha learned PP for the State This appeal is directed against the judgment and order of conviction and sentence dated 27072015passed by the learned Sessions Judge Lohardaga in Sessions Trial No 44 of 2013 whereby and whereunder the appellants have been convicted for the offences punishable under Sections 302 34 and 201 34 of IPC and have been sentenced to undergo SI for life and a fine of Rs 10 000 for the offence under Section 302 34 of IPC No separate sentence has been passed for the offence under Section 201 34 of IPC The prosecution story arising out of an FIR instituted by Jairam Munda is to the effect that the son of the informant had gone to his in laws place on 05022013 at village Panchpadwa in the district of Lohardaga On 06022013 at about 8 AM he came to know that his son has been murdered by Boda Munda  Suman Munda Birsa Munda  Runka Munda Jama Munda  Sushil Munda and Arti Kumari  Sukar Kumari At this he along with his another son reached Panchpadwa where he found his son having been murdered and thrown at Panchpadwa Tongri The reason for the occurrence is that the son of the 2 informant Mansukh Munda used to remain sick and in spite of being treated at various places he was not getting well It has been alleged that the informant and his family members were thinking of taking Mansukh Munda to Devas for treatment and this intention was disclosed to his brother Hadu Munda and on this issue there appeared a fissure in the relationship between the families of the informant and Hadu Munda It has been alleged that the family of Hadu Munda left for Panchpadwa The informant had sent his son to bring them back but they did not return On 28012013 the son of the informant Markas Munda had died and there was friction between both the sides for practicing witchcraft which resulted in the occurrence The informant could came to know that his son Markas Munda had gone to the house of Bauda Munda on 05022013 and near the door stop Arti Kumari had started assaulting him with fists slaps and danda It has been alleged that all the accused persons had dragged Markas Munda towards the jungle and assaulted him on his head with lathi and stones which resulted in his death It has also been stated that his relation Baura Munda has seen the occurrence Based on the aforesaid allegations Jobang P S Case No 2 of 2013 was instituted under Section 302  201 34 of IPC in which after investigation charge sheet was submitted leading to taking of cognizance and after the case was committed to the court of Sessions charge was framed for the offences punishable under Sections 302 34  201 34 of IPC and under Section 3 of the Prevention of Witch Practices Act 1999 to which the accused pleaded not guilty and claimed to be tried In course of trial 7 witnesses were examined by the prosecution PW 1 Boda Munda  Bauda Munda has stated that the incident is of 6 months back He had gone to Kisko to prepare his Aadhar Card and he returned back to his house from Kisko at about 8 PM He has stated that while returning home he has seen his son in law Markas Munda being assaulted with lathi and danda by 4 5 persons He has further stated that after assaulting him he was dragged towards the forest where he was hit by a big stone leading to his death He has further stated that the persons involved are Birsa Munda  Runka Munda Bora Munda  Suman Munda Jama Munda  Sushil Munda PW 2 Sohrai Munda has stated that on the date of occurrence 3 and Aarti Kumari  Sukar Kumari He has stated that thereafter on account of fear he had gone to his residence In cross examination this witness has stated that it was dark when the incident had taken place He has further deposed that due to darkness persons who were standing at a distance could not be recognized He has also stated that he does not know as to when his son in law had come to his house After coming from Kisko he had straightway gone to his house He has further stated that all the accused are from his village with whom this witness is not on very friendly terms Replying to a court question this witness has stated that he was at a distance of 150 yards from the place of occurrence he along with PW 1 was returning from Kisko to their respective homes On reaching the village they heard the sound of shouting and when they reached near the house of PW 1 they had seen Markas Munda being assaulted by the accused persons He has further stated that the accused persons took Markas Munda towards Tongri and thereafter Markas Munda was assaulted on his head with a big stone and he and PW 1 fled away from the place of occurrence and reached their respective homes He has stated that on the next date the father of Markas Munda was informed In cross examination this witness has stated that it was total darkness at the time of returning home from Kisko Block He has admitted that there is no electricity in his village He has further stated that while entering his village his house is situated before the house of PW 1 He has further stated that the incident was never disclosed by them to the villagers He has also deposed that he had not seen any person in the place of occurrence occurrence he was in the house and on next date Sohrai Munda informed on phone that his elder brother has been done to death He has also stated that on hearing such information he had gone to village Panchpadwa and had seen the dead body of his brother Sohrai had disclosed to him about the accused persons committing the murder of his brother He has also stated that the reason for the occurrence is that Devas had told him that it was on account of his uncle Hadu Munda that his brother Mansukh Munda was sick PW 3 Ram Nath Munda has stated that on the date of 4 PW 4 Dr Pranav Kumar was posted as a Medical Officer on 07022013 in the Department of Leprosy where he conducted autopsy on the dead body of Markas Munda This witness has found the following External Injuries Lacerated wound on left side of forehead size 2 inch x 1 inch x up to bone deep Fracture of left mandible iii Extraction of upper four teeth iv Blood stain on nose and ear Internal Injuries A Head and Neck Brain matter lacerated and pale on section B Thoracic cavity Lungs intact and pale on section heart Intact right chamber full and left chamber empty C Abdominal cavity Liver spleen and kidneys were intact and pale on cut section Stomach contains semi digested food particle and gas large intestine contains faecal matter All above injuries were ante morten in nature and time elapsed since death 6 to 36 hours The cause of death according to this witness was due to Haemorrhage and shock caused by hard and blunt substance This witness has proved the post mortem report which has been marked as Exhibit 1 PW 5 Dedo Munda is a hearsay witness PW 6 Jairam Munda is the informant who has stated that on the date of occurrence he was at his home and his son Markas Munda had gone to his in laws place at village Panchpadwa He has also stated that his son had gone to call the brother of the informant namely Hadu Munda He has also stated that his son was assaulted and done to death by the accused persons The reasons for the occurrence according to this witness was that he and his brother were not on good terms In cross examination it has been stated that there was no enmity with the accused Bora Munda Jama Munda and Birsa Munda and they were on friendly terms with him 5 It has been stated by the learned counsel for the appellants that the 11 PW 7 Supa Kongari has deposed that on 06022013 he was posted as Sub Inspector on Jobang Police Station He has recorded the Fard Beyan of Jairam Munda which has been proved and marked as Exhibit 2 The formal FIR is in his hand writing and signature which has been marked as Exhibit 3 He has further stated that after taking over investigation he had recorded the re statement of the informant and had thereafter taken the statement of Baura Munda Sohrai Munda Dedo Munda and Ram Nath Munda This witness has inspected the place of occurrence which is at village Panchpadwa Tongri which is an isolated place surrounded by jungle and bushes On 17022013 he had obtained the post mortem report and on direction of his superior officials he had submitted charge sheet In cross examination this witness has stated that he had not seized any article during investigation He has also stated that the house of Bauda Munda is at a distance of half km from the place of occurrence On a court question he has stated that inquest report has been misplaced and he also could not recover the big stone which was used in the commission of the murder prosecution has miserably failed to prove its case beyond all reasonable doubt Learned counsel submits that save and except PW 1 and 2 there are no eye witnesses to the occurrence and the evidence of PW 1 and PW 2 suffers from major discrepancies which itself makes the identification of the appellants as the perpetrators of the offence doubtful Learned counsel submits that the PW 1 and PW 2 had stated that it was a dark night and there is no electricity in the village and as such it would have been absolutely impossible to have identified the assailants Learned counsel further while referring to the evidence of PW 1 submits that the conduct of PW 1 is improbable and unbelievable if his version is taken to be true to the effect that he claims to have seen the accused persons committing the murder of his son in law but neither had he tried to save him nor did he inform this matter to any of the villagers immediately Learned counsel also submits that as per PW 2 the incident is said to have taken place near the house of PW 1 but the IO has disclosed that the distance between the place of occurrence and the house of PW 1 is about half a km He further submits that nothing 6 has been stated as to whether PW 1 and PW 2 had followed them which further nullifies the story created by PW 1 and PW 2 to implicate the appellants Learned counsel submits that as per PW 1 both the sides were on inimical terms and hence the false implication of the appellant cannot be ruled out On such parameters learned counsel for the appellants has stated that the present appeal be allowed 13 Mr Shekhar Sinha learned PP for the State on the other hand while relying on the evidence of PW 1 and PW 2 has stated that they are the eye witnesses of the occurrence and their evidence is cogent convincing and believable and therefore conviction of the appellants under Section 302 34 and 201 34 of IPC has rightly been done He therefore prays that the present appeal be dismissed 14 We have heard the rival submissions and have also gone through the Lower Court Records The prosecution has tried to build an edifice based on the evidence of PW 1 and PW 2 who have stated to have seen the occurrence relating to the son of the informant namely Markas Munda being subjected to assault by the appellants with stone and tangi However the edifice crumbles once the cross examination of PW 1 and PW 2 is taken into consideration PW 1 in very categorical terms has stated in his cross examination that it was difficult for the persons to be recognized who were standing at a distance on account of the darkness He has further admitted that it was a dark night According to the evidence of PW 1 he out of fear had returned back to his residence which conduct appears to be totally contrary to what one would expect in such a situation when the son in law of the PW 1 is being done to death and he neither informs the villagers nor raises any alarm but rushes to his house and stays put On a court question he has stated that from the place of occurrence he was at a distance of 150 feet and considering the darkness which had engulfed the area and the distance from where the incident is said to have been seen by PW 1 it totally nullifies the prosecution case with respect to the identification of the appellants being the assailants Similarly the evidence of PW 2 is fraught with major discrepancies as he has stated that he had not seen any 7 person in the place of occurrence and that he had never disclosed the incident to the villagers This witness has also admitted that there is no electricity in the village He has also stated about total darkness when he was returning home from Kisko Block The identification of the appellants by PW 1 also is nullified if the distance between the place of occurrence and the house of PW 1 is taken into consideration as disclosed by PW 7 the Investigating Officer which is about a half km and coupled with the darkness the identification of the appellants by PW 2 therefore seems to be an absurd story created by PW 2 It is to be noted herein that PW 1 has stated about he being on inimical terms with the appellants and therefore the false implication of the appellants also cannot be ruled out The Investigating Officer has failed to seize the weapon of assault or the stone with which the deceased was purportedly done to death The evidence of the other witnesses do not energise the prosecution case and the falsity of the allegations levelled against the appellants lie threadbare on a dissection of the evidence of the so called eye witnesses PW 1 and PW 2 as has been taken note of earlier The prosecution therefore having miserably failed to prove its case beyond all reasonable doubt the appellants deserves to be acquitted from the charges against them In such circumstances this appeal stands allowed and the impugned judgment and order of conviction and sentence dated 27072015 and 31072015 respectively passed by the learned Sessions Judge Lohardaga in Sessions Trial No 413 are hereby set aside Since the appellant no 1 is on bail she is discharged from the liability of her bail bonds As regards appellant nos 2 to 4 are concerned they are directed to be released immediately and forthwith if not wanted in any other case This appeal stands allowed Rongon Mukhopadhyay J Rajesh Kumar J Jharkhand High Court at Ranchi The 28th day of July 2021 RShekhar NAFR Cp3"  # Replace with your input

# Split into manageable chunks
text_chunks = split_text_with_token_limit(input_text, tokenizer)

# File paths for saving document vectors and attention weights
vector_file_path = "document_vectors.npy"
attention_file_path = "attention_weights.txt"

# Generate summaries and save document vectors and attention weights
summaries = save_and_print_vectors_and_attention(text_chunks, model, tokenizer, vector_file_path, attention_file_path)

# Combine summaries
final_summary = ' '.join(summaries)
print("Final Summary:", final_summary)

# Print the saved document vectors and attention weights
print_document_vectors(vector_file_path)
print_attention_weights(attention_file_path)


Token indices sequence length is longer than the specified maximum sequence length for this model (3476 > 512). Running this sequence through the model will result in indexing errors


Streaming output truncated to the last 5000 lines.
 1.8105839e-03 2.0057135e-03 1.1151762e-03 1.3190421e-03 1.4162490e-03
 1.9516795e-03 1.3180141e-03 1.1263910e-03 7.3685264e-04 7.0664793e-04
 8.4853749e-04 7.7673779e-03 2.6172947e-03 4.3714475e-03 1.1877948e-02
 4.7287876e-03 1.6935194e-01]
Token: ▁imp, Attention Score: [6.12027769e-04 7.89630052e-04 4.34890419e-04 4.79869050e-04
 8.93521763e-04 6.51374459e-04 5.66156174e-04 4.31802968e-04
 4.87580895e-04 5.27373340e-04 2.03825868e-04 1.08287386e-04
 1.70663130e-04 2.07663921e-04 3.32960335e-04 6.66414795e-04
 4.83504817e-04 8.74502934e-04 1.26960676e-03 3.51943891e-03
 9.86795290e-04 4.23661433e-04 9.35439952e-04 1.19257730e-03
 1.33420201e-03 2.65446445e-03 5.01715112e-04 1.64813059e-03
 3.39922466e-04 4.81113093e-04 1.27736886e-04 3.62783525e-04
 2.74829054e-03 6.92322676e-04 2.68838776e-04 9.28695488e-04
 2.84274444e-02 9.01982712e-04 1.48429268e-03 4.12173686e-04
 6.11911004e-04 8.25753261e-04 1.33375218e-03 8.96381447e-04
 8.75

In [30]:
import numpy as np
from transformers import T5ForConditionalGeneration, AutoTokenizer

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained("./legal_summarizer")
tokenizer = AutoTokenizer.from_pretrained("./legal_summarizer")

# Enable output of hidden states and attention weights
model.config.output_hidden_states = True
model.config.output_attentions = True  # Enable attention outputs

# Split text into token-limited chunks
def split_text_with_token_limit(text, tokenizer, max_tokens=512, overlap=50):
    tokens = tokenizer(text, return_tensors="pt", truncation=False)["input_ids"][0]
    chunks = []
    for i in range(0, len(tokens), max_tokens - overlap):
        chunk_tokens = tokens[i:i + max_tokens]
        chunks.append(tokenizer.decode(chunk_tokens, skip_special_tokens=True))
    return chunks

# Save words with attention weights and print the document vector and attention scores
def save_and_print_vectors_and_attention(text_chunks, model, tokenizer, vector_file_path, attention_file_path):
    all_vectors = []
    all_attention_weights = []  # To store attention weights
    summaries = []

    for idx, chunk in enumerate(text_chunks):
        # Tokenize chunk
        inputs = tokenizer(chunk, return_tensors="pt", truncation=True, padding=True)

        # Forward pass through the encoder to extract hidden states and attention weights
        outputs = model.encoder(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            return_dict=True,
            output_hidden_states=True,
            output_attentions=True,  # Get attention weights
        )

        # Extract the encoder's last hidden state and attention weights
        document_vector = outputs.last_hidden_state.mean(dim=1).detach().cpu().numpy()  # Mean pooling
        all_vectors.append(document_vector)

        # Extract attention weights from the output
        attentions = outputs.attentions  # List of attention weights for each layer
        # We'll use the attention from the last layer for simplicity
        last_attention = attentions[-1]  # (batch_size, num_heads, seq_len, seq_len)

        # Aggregate attention scores across all attention heads
        attention_scores = last_attention.mean(dim=1).squeeze().detach().cpu().numpy()  # Average across heads

        # Map tokens to words and align with attention scores
        tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])  # Convert token ids to words/subwords
        word_attention_map = {}
        word_start = 0

        # Reconstruct words from subwords and aggregate attention scores
        for i, token in enumerate(tokens):
            word = tokenizer.convert_tokens_to_string([token])  # Get word/subword back
            if word.startswith('##'):
                word = word[2:]  # Remove subword prefix
            if word not in word_attention_map:
                word_attention_map[word] = attention_scores[i]
            else:
                word_attention_map[word] += attention_scores[i]  # Aggregate attention for split words

        all_attention_weights.append(word_attention_map)

        # Generate summary using the full model
        summary_ids = model.generate(
            input_ids=inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=256,
            min_length=50,
            do_sample=False,
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)

        print(f"Processed chunk {idx + 1}/{len(text_chunks)}")

    # Save all document vectors to a file
    np.save(vector_file_path, np.vstack(all_vectors))  # Stack and save all vectors
    print(f"Document vectors saved to {vector_file_path}")

    # Save the attention weights to a text file
    with open(attention_file_path, 'w') as f:
        for idx, attention_map in enumerate(all_attention_weights):
            f.write(f"Attention for chunk {idx + 1}:\n")
            for word, score in attention_map.items():
                f.write(f"Word: {word}, Attention Score: {score}\n")
            f.write("\n")

    return summaries

# Print the content of the saved document vector file
def print_document_vectors(vector_file_path):
    print("\nDocument Vectors:")
    vectors = np.load(vector_file_path)
    print(vectors)

# Print the content of the saved attention weights file
def print_attention_weights(attention_file_path):
    print("\nAttention Weights:")
    with open(attention_file_path, 'r') as f:
        print(f.read())

# Input text
input_text = "Criminal AppealNo 10115 Against the judgment and order of conviction and sentence dated 27072015 by the learned Sessions Judge Lohardaga in Sessions Trial No 413 1Arti  Aarti  Sukar Kumari 2Jama  Sushil Munda 3Birsa Munda  Runka Munda 4Boda Munda  Suman Munda   Appellants The State of Jharkhand For the Appellant For the Respondent Mr Birendra Kumar Advocate Mr Vijay Kumar Sinha Advocate Mr Shekhar Sinha PP Present HON BLE MR JUSTICE RONGON MUKHOPADHYAY HONBLE MR JUSTICE RAJESH KUMAR Heard Mr Birendra Kumar learned counsel for the appellants and Mr Shekhar Sinha learned PP for the State This appeal is directed against the judgment and order of conviction and sentence dated 27072015passed by the learned Sessions Judge Lohardaga in Sessions Trial No 44 of 2013 whereby and whereunder the appellants have been convicted for the offences punishable under Sections 302 34 and 201 34 of IPC and have been sentenced to undergo SI for life and a fine of Rs 10 000 for the offence under Section 302 34 of IPC No separate sentence has been passed for the offence under Section 201 34 of IPC The prosecution story arising out of an FIR instituted by Jairam Munda is to the effect that the son of the informant had gone to his in laws place on 05022013 at village Panchpadwa in the district of Lohardaga On 06022013 at about 8 AM he came to know that his son has been murdered by Boda Munda  Suman Munda Birsa Munda  Runka Munda Jama Munda  Sushil Munda and Arti Kumari  Sukar Kumari At this he along with his another son reached Panchpadwa where he found his son having been murdered and thrown at Panchpadwa Tongri The reason for the occurrence is that the son of the 2 informant Mansukh Munda used to remain sick and in spite of being treated at various places he was not getting well It has been alleged that the informant and his family members were thinking of taking Mansukh Munda to Devas for treatment and this intention was disclosed to his brother Hadu Munda and on this issue there appeared a fissure in the relationship between the families of the informant and Hadu Munda It has been alleged "  # Replace with your input

# Split into manageable chunks
text_chunks = split_text_with_token_limit(input_text, tokenizer)

# File paths for saving document vectors and attention weights
vector_file_path = "document_vectors.npy"
attention_file_path = "attention_weights.txt"

# Generate summaries and save document vectors and attention weights
summaries = save_and_print_vectors_and_attention(text_chunks, model, tokenizer, vector_file_path, attention_file_path)

# Combine summaries
final_summary = ' '.join(summaries)
print("Final Summary:", final_summary)

# Print the saved document vectors and attention weights
print_document_vectors(vector_file_path)
print_attention_weights(attention_file_path)


Token indices sequence length is longer than the specified maximum sequence length for this model (553 > 512). Running this sequence through the model will result in indexing errors


Streaming output truncated to the last 5000 lines.
 1.32504647e-04 1.23451682e-04 6.62313978e-05 9.63819766e-05
 1.10401212e-04 1.00658130e-04 9.83004647e-05 1.26815590e-04
 6.02953041e-05 5.04191521e-05 8.72663877e-05 4.79745737e-04
 1.21091114e-04 5.82721026e-04 9.46484579e-05 1.18067648e-04
 9.58470991e-05 1.27244057e-04 4.22895362e-04 3.25532455e-04
 4.99653746e-04 5.16459870e-04 7.57830264e-03 9.04320841e-05
 1.49829066e-04 1.35472961e-04 2.21432652e-04 7.70295519e-05
 1.22165700e-04 1.34881586e-04 2.43848044e-04 7.49471437e-05
 7.92917854e-05 1.39060590e-04 1.53813817e-04 1.41271885e-04
 8.91231466e-05 1.70765634e-04 1.40741584e-04 9.97132520e-05
 2.71676807e-04 2.41072872e-03 7.55861867e-04 2.47058866e-04
 9.63469938e-05 1.06196385e-04 1.49505315e-04 1.49284475e-04
 1.01952188e-04 7.95794404e-05 6.27623158e-05 9.96225135e-05
 1.37877316e-04 9.80912300e-05 7.83026844e-05 6.09891031e-05
 3.80680540e-05 1.03753628e-04 1.20955126e-04 1.82241332e-04
 1.67102888e-04 5.15177206e-04 8.4